In [1]:
import pandas as pd
import numpy as np
import os
import string
import re
import emoji
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import backend as K

cwd = os.getcwd()
train_clean_set = pd.read_csv(cwd + "/train_clean_set.csv")
test = pd.read_csv(cwd + "/test_clean_set.csv")

Using TensorFlow backend.


In [2]:
train_clean_set.head()

,search_text,label,word_count,length,avg_word
0,áo khoác adidas nu,1,4,18,3.750000
1,mỗi thùng bia đều có thẻ cào hả chị,0,9,35,3.000000
2,hàng mới về váy đũi nhún eo 3 màu y hình 270k,1,12,47,2.833333
3,polo adidas performance bayer 100 cotton ạ si...,1,18,88,3.833333
4,adidas uk 540k cân xanh size 1824m hồng size ...,1,10,50,4.000000


In [3]:
train_clean_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
search_text    39999 non-null object
label          40000 non-null int64
word_count     40000 non-null int64
length         40000 non-null int64
avg_word       40000 non-null float64
dtypes: float64(1), int64(3), object(1)
memory usage: 1.5+ MB


In [4]:
train_clean_set.describe()

,label,word_count,length,avg_word
count,40000.000000,40000.00000,40000.000000,40000.000000
mean,0.501225,122.48765,607.335975,3.696773
std,0.500005,331.87957,2137.644353,1.128915
min,0.000000,0.00000,0.000000,0.000000
25%,0.000000,34.00000,165.000000,3.253012
50%,1.000000,63.00000,297.500000,3.500000
75%,1.000000,106.00000,511.000000,3.837838
max,1.000000,19898.00000,172395.000000,64.333333


In [5]:
train_clean_set = train_clean_set[train_clean_set['length'] <= 1000]
X = train_clean_set.search_text.astype(str)
Y = train_clean_set.label
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1)

In [6]:
train_clean_set.describe()

,label,word_count,length,avg_word
count,35982.000000,35982.000000,35982.000000,35982.000000
mean,0.480852,64.809349,308.385832,3.668434
std,0.499640,45.440254,218.817313,0.998909
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,31.000000,145.000000,3.230769
50%,0.000000,56.000000,266.000000,3.491935
75%,1.000000,90.000000,423.000000,3.847029
max,1.000000,240.000000,1000.000000,64.333333


In [7]:
max_words = 20000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [8]:
print(tok.texts_to_sequences(['hello']))

[[4375]]


In [9]:
def RNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [10]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred,0,1)))
    possible_positives = K.sum(K.round(K.clip(y_true,0,1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred,0,1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred,0,1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_score_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    f1_score = 2 * (precision*recall) / (precision+recall)
    return f1_score

In [11]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy', recall_m, precision_m, f1_score_m])

W0717 16:09:07.660923 140490256217920 deprecation_wrapper.py:119] From /home/ngtamdat/anaconda3/envs/jnb/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 16:09:07.687791 140490256217920 deprecation_wrapper.py:119] From /home/ngtamdat/anaconda3/envs/jnb/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 16:09:07.688991 140490256217920 deprecation_wrapper.py:119] From /home/ngtamdat/anaconda3/envs/jnb/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 16:09:07.903767 140490256217920 deprecation_wrapper.py:119] From /home/ngtamdat/anaconda3/envs/jnb/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is d

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
__________

In [12]:
model.fit(sequences_matrix, Y_train, batch_size=128,epochs=10,validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 29144 samples, validate on 3239 samples
Epoch 1/10
29144/29144 [==============================] - 23s 794us/step - loss: 0.3203 - acc: 0.8684 - recall_m: 0.8733 - precision_m: 0.8555 - f1_score_m: nan - val_loss: 0.2627 - val_acc: 0.8975 - val_recall_m: 0.9544 - val_precision_m: 0.8537 - val_f1_score_m: 0.9008
Epoch 2/10
14464/29144 [=============>................] - ETA: 9s - loss: 0.1947 - acc: 0.9300 - recall_m: 0.9327 - precision_m: 0.9211 - f1_score_m: 0.9258

KeyboardInterrupt: 

In [ ]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
accr = model.evaluate(test_sequences_matrix,Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Recall: {:0.3f}\n  Precision: {:0.3f}\n  F1 Score: {:0.3f}'.format(accr[0],accr[1],accr[2],accr[3], accr[4]))

In [ ]:
X_test = test.search_text.astype(str)
Y_test = test.label

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

accr = model.evaluate(test_sequences_matrix, Y_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}\n  Recall: {:0.3f}\n  Precision: {:0.3f}\n  F1 Score: {:0.3f}'.format(accr[0],accr[1],accr[2],accr[3], accr[4]))

In [ ]:
test_predict = sequence.pad_sequences(tok.texts_to_sequences(['số điện thoại của tôi là']),maxlen=max_len)
model.predict(test_predict)

In [ ]:
test.describe()